<a href="https://colab.research.google.com/github/HedersonSantos/Noticias/blob/main/preProcessamento_noticias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import numpy as np
from google.colab import files, drive
from pathlib import Path
from sklearn.model_selection import train_test_split
import io
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', None)

### Montando drive Google e setando variáveis

In [14]:
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/Colab\ Notebooks/nlp_tcc
path = '/gdrive/My Drive/Colab Notebooks/nlp_tcc'

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/Colab Notebooks/nlp_tcc


### Upload da base de noticias

In [15]:
#csvNoticias = files.upload()  
!rm *.*
!wget https://raw.githubusercontent.com/HedersonSantos/Noticias/main/noticia_tratada.zip
!unzip noticia_tratada.zip
%ls



--2021-06-21 19:34:28--  https://raw.githubusercontent.com/HedersonSantos/Noticias/main/noticia_tratada.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25634699 (24M) [application/zip]
Saving to: ‘noticia_tratada.zip’

noticia_tratada.zip 100%[===================>]  24.45M  37.7MB/s    in 0.6s    

2021-06-21 19:34:29 (37.7 MB/s) - ‘noticia_tratada.zip’ saved [25634699/25634699]

Archive:  noticia_tratada.zip
  inflating: noticia_tratada.csv     
bertimbau_classifier_pt_dict  noticia_tratada.csv  noticia_tratada.zip


In [16]:
#dfNews_all = pd.read_csv(io.BytesIO(csvNoticias['noticia_tratada.csv']), header=0, delimiter=';')
dfNews_all = pd.read_csv('noticia_tratada.csv', header=0, delimiter=';')
print(dfNews_all.columns)
dfNews_all.shape

Index(['FONTE', 'CATEGORIA', 'TEXTO', 'URL', 'PUBLISHEDAT', 'TITLE', 'AUTHOR',
       'DT_REGISTRO', 'TEXT_TRATADO'],
      dtype='object')


(15799, 9)

### Tratamento da CATEGORIA

In [17]:
print('Dimensao Inicial:',dfNews_all.shape)
#Elimina noticias duplicadas
dfNews = dfNews_all.drop_duplicates(subset=['URL'],keep='first').copy()
#Elimina noticias sem texto
dfNews = dfNews[(~dfNews['TEXT_TRATADO'].isnull())]
print('Dimensão após eliminar duplicadas e textos em branco:',dfNews.shape)
#Tira as marcações do campo Text[] do Postgres em categoria e seta para minúscula
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: x['CATEGORIA'].replace('[','').replace(']','').replace('"','').replace(", 'redacao'",'').replace("'",''),axis=1)
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: x['CATEGORIA'].lower(),axis=1)
#Agrupa as notícias locais como categoria LOCAL
siglaEstados = ['ac','al','ap','am','ba','ce','df','es','go','ma','mt','ms','mg','pa','pb','pr','pe','pi','rj','rn','rs','ro','rr','sc','sp','se','to']
nomeEstados = ['acre','alagoas','amapa','amazonas','bahia','ceara','distrito-federal','espirito-santo','goias','maranhao','mato-grosso','mato-grosso-sul','minas-gerais','minas',
               'para','paraiba','parana','pernambuco','piaui','rio-janeiro','rio-grande-norte','rio-grande-sul','rondonia','roraima','santa-catarina','sao-paulo','sergipe','tocantins']
nomeCidades =['rio','bh','brasilia']
local = siglaEstados + nomeEstados + nomeCidades
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'local' if x['CATEGORIA'] in local else x['CATEGORIA'],axis=1)
#efe, afp, rfi são notícias internacionais. 
mundo = ['efe','afp','rfi','mundo','internacional']
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'mundo' if any( i in x['CATEGORIA'] for i in mundo) else x['CATEGORIA'],axis=1)
#Na lista miscelanea há as categorias que possuem miscelâneas de noticias de outras categorias.
miscelanea = ['deutschewelle', '','bbc','ansa','universa']
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'miscelanea' if x['CATEGORIA'] in miscelanea else x['CATEGORIA'],axis=1)
#'reuters' é uma categoria especifica do UOL. Possui textos de economia e miscelânea de noticias.
dfreuters = dfNews[dfNews['CATEGORIA']=='reuters'].copy()
dfreuters['CATEGORIA'] = dfreuters.loc[:,['CATEGORIA','URL']].apply(lambda x: 'economia' if 'economia' in x['URL']  else 'miscelanea',axis=1)
dfNews=dfNews[dfNews['CATEGORIA']!='reuters']
dfNews = pd.concat([dfNews,dfreuters],sort=False)
#noticias é uma categoria da globo. Traz noticias do Jornal Nacional. São miscelâneas.
dfnoticia = dfNews[dfNews['CATEGORIA']=='noticia'].copy()
dfnoticia['CATEGORIA'] = dfnoticia.loc[:,['CATEGORIA','URL']].apply(lambda x: 'saude' if 'ciencia-e-saude' in x['URL']  else 'miscelanea',axis=1)
dfNews=dfNews[dfNews['CATEGORIA']!='noticia']
dfNews = pd.concat([dfNews,dfnoticia],sort=False)
#redacao é uma categoria da uol. Traz noticias do meio-ambiente. VERIFICAR QUANDO ATUALIZAR O DATASET SE PERMANECE ASSIM.
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA','URL']].apply(lambda x: 'meio-ambiente' if ('meio-ambiente' in x['URL']) and
                                                              (x['CATEGORIA']=='redacao') else x['CATEGORIA'],axis=1)
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA','URL']].apply(lambda x: 'meio-ambiente' if ('meio-ambiente' in x['URL']) and
                                                              (x['CATEGORIA']=='sociedade') else 
                                                              'miscelanea' if x['CATEGORIA']=='sociedade' else x['CATEGORIA'] ,axis=1)
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'saude' if any(i in x['CATEGORIA'] for i in ['vivabem','bemestar'])  else x['CATEGORIA'] ,axis=1)
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'esporte' if any(i in x['CATEGORIA'] for i in ['futebol','volei']) else x['CATEGORIA'],axis=1)
midia = ['famosos','cinema','oscar','videos','bbb','musica','lives','cultura']
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'midia' if any(i in x['CATEGORIA'] for i in midia) else x['CATEGORIA'],axis=1)
dfNews['CATEGORIA'] = dfNews.loc[:,['CATEGORIA']].apply(lambda x: 'politica' if any(i in x['CATEGORIA'] for i in ['politica'])  else x['CATEGORIA'],axis=1)
dfNews.shape

Dimensao Inicial: (15799, 9)
Dimensão após eliminar duplicadas e textos em branco: (15799, 9)


(15799, 9)

In [ ]:
#Exportando o dataset todo para csv
#dfNews.to_csv(path + "/news.csv",header=True,doublequote=True, index=False, quotechar='"', encoding='utf-8')

In [18]:
dfNews.groupby(['CATEGORIA'])['URL'].count().reset_index().sort_values('URL',ascending=False)

,CATEGORIA,URL
396,miscelanea,1833
395,midia,1574
263,brasil,1381
439,politica,1337
386,local,1216
400,mundo,1115
339,esporte,720
461,saude,665
416,novelas,628
317,economia,607


In [ ]:
#Filtra categorias com poucas noticias. 
'''dfNewsGrupo = dfNews.groupby(['CATEGORIA'])['URL'].count().reset_index().sort_values('URL',ascending=False)
dfNewsGrupo = dfNewsGrupo[dfNewsGrupo['URL']<=30]
dfNews = dfNews[~dfNews['CATEGORIA'].isin(dfNewsGrupo['CATEGORIA'].tolist())]
dfNews.shape '''

(6992, 9)

In [ ]:
#Criando coluna CATEGORIA numerada.
'''dfNewsGrupo = dfNews.groupby(['CATEGORIA'])['URL'].count().reset_index().sort_values('URL',ascending=False)
print('QUANTIDADE DE NOTICIAS POR CATEGORIA:\n',dfNewsGrupo)
dictCategoriaNumero = dict(zip(dfNewsGrupo['CATEGORIA'],range(0,30)))
print(dictCategoriaNumero)
dfNews['CAT_NRO'] = dfNews['CATEGORIA'].map(dictCategoriaNumero)
dfNews[['CATEGORIA','CAT_NRO']].head(5)
'''

QUANTIDADE DE NOTICIAS POR CATEGORIA:
         CATEGORIA   URL
11          midia  1164
9           local   979
16       politica   906
13          mundo   731
14        novelas   552
12     miscelanea   491
17          saude   390
6         esporte   378
5        economia   301
0          brasil   270
3     coronavirus   226
2         colunas   183
8         justica   129
1    celebridades    85
7      fantastico    47
10  meio-ambiente    45
4       cotidiano    45
18     tecnologia    39
15        podcast    31
{'midia': 0, 'local': 1, 'politica': 2, 'mundo': 3, 'novelas': 4, 'miscelanea': 5, 'saude': 6, 'esporte': 7, 'economia': 8, 'brasil': 9, 'coronavirus': 10, 'colunas': 11, 'justica': 12, 'celebridades': 13, 'fantastico': 14, 'meio-ambiente': 15, 'cotidiano': 16, 'tecnologia': 17, 'podcast': 18}


,CATEGORIA,CAT_NRO
0,midia,0
1,midia,0
2,miscelanea,5
3,mundo,3
4,local,1


# Tratamento dos Textos


In [28]:
dfNews.groupby('FONTE')['URL'].count().reset_index().sort_values(by='URL', ascending=False)


,FONTE,URL
32,UOL,2799
38,google-news-br,2211
1,Brasil 247,1915
29,Poder360,956
21,Notícias da TV,813
23,O Globo,780
10,Folha de S.Paulo,518
26,O Tempo,439
16,Jornal Folha Vitória,399
17,Jornal de Brasília,354


In [30]:
dfN = dfNews[dfNews['FONTE']=='UOL']
dfN=dfN[0:5]
for t in dfN['TEXTO']:
  print(t)
  print('------------------------------------------------------------------------------------------------')
  break

Do UOL, em São Paulo
Do UOL, em São Paulo
25/04/2021 11h17Atualizada em 25/04/2021 11h17 
A ex-ministra do Meio Ambiente Marina Silva recebeu a primeira dose da vacina contra a covid-19 ontem, no Distrito Federal. Com 63 anos, ela integra a faixa etária contemplada pela campanha de imunização da capital do país.
Muito feliz porque, dentro dos critérios da faixa de idade do Distrito Federal, pude tomar a primeira dose da vacina contra a covid-19. Mais uma vez o serviço público de saúde exerce papel fundamental em minha vida. Marina Silva, ex-ministra do Meio Ambiente
Marina afirmou que segue na "luta" para manter um sistema público de saúde de qualidade para todos e enalteceu a ciência.
A ex-ministra que concorreu nas eleições para Presidente da República publicou uma foto com o cartão de vacinação preenchido nas mãos e um vídeo em que, após receber a dose do imunizante, ela comemora: "Viva ao SUS".
Muito feliz porque, dentro dos critérios da faixa de idade do Distrito Federal, pude tom

### Prepara DataSet Treinamento, Teste

In [ ]:
dfMiscelanea = dfNews[dfNews['CATEGORIA']=='miscelanea'].loc[:,['TEXT_TRATADO']]
print('miscelanea:',dfMiscelanea.shape)



miscelanea: (491, 1)


In [ ]:
dfNews.columns

Index(['FONTE', 'CATEGORIA', 'TEXTO', 'URL', 'PUBLISHEDAT', 'TITLE', 'AUTHOR',
       'DT_REGISTRO', 'TEXT_TRATADO', 'CAT_NRO'],
      dtype='object')

In [ ]:
#vamos obter uma amostra randômica de n noticias para cada CATEGORIA e dividir em três datasets Treinamento Validação e Teste.
dfNewsGrupo = dfNewsGrupo[dfNewsGrupo['CATEGORIA']!='miscelanea']
n=200
categorias = dfNewsGrupo[dfNewsGrupo['URL']>n].loc[:,['CATEGORIA']]['CATEGORIA'].to_list()
print(categorias)
frame=[]
for categoria in categorias:
  dftmp = dfNews[dfNews['CATEGORIA']==categoria].loc[:,['TEXTO','CAT_NRO']].sample(n=n)
  frame.append(dftmp)
dfAmostra = pd.concat(frame,ignore_index=True, sort=False)
print('Amostra:', dfAmostra.shape)

df_train, df_test_valid = train_test_split(dfAmostra, train_size = 0.8, random_state = 10)
df_valid, df_test = train_test_split(df_test_valid, train_size = 0.5, random_state = 10)
print('Treinamento:',df_train.shape)
print('Teste:', df_test.shape)
print('Valid:', df_valid.shape)



['local', 'politica', 'mundo', 'novelas', 'saude', 'esporte', 'economia', 'brasil']
Amostra: (1600, 2)
Treinamento: (1280, 2)
Teste: (160, 2)
Valid: (160, 2)


In [ ]:
df_train.to_csv(path + "/train.csv",header=True,doublequote=True, index=False, quotechar='"', encoding='utf-8')
df_test.to_csv(path + "/test.csv",header=True,doublequote=True, index=False, quotechar='"', encoding='utf-8')
df_valid.to_csv(path + "/valid.csv",header=True,doublequote=True, index=False, quotechar='"', encoding='utf-8')